In [1]:
## 만들던 거

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

# ==============
# A. 데이터 로드
# ==============

df = pd.read_csv('Merged_Resistivity_Hardness_Data.csv') # 파일 로드
#print(f"df : {df}")
Y_cols = ['Resistance', 'Hardness'] # 예측할 종속변수 ()
#print(f"Y_cols : {Y_cols}")
X_exclude = ['Number', 'Thickness'] + Y_cols # 제외할 목록(비저항과 경도를 병합하면서 생김)
#print(f"X_exclude : {X_exclude}")
X_cols = [col for col in df.columns if col not in X_exclude] # 실제로 사용할 목록
print(f"X_cols : {X_cols}")
X_temp = df[X_cols] # 제외한 목록을 뺀 다른 목록들
#print(f"X_temp : {X_temp}")
Y_temp = df[Y_cols] # 비저항, 경도 값만 있는 목록
#print(f"Y_temp : {Y_temp}")
X = X_temp.select_dtypes(include=np.number).fillna(0) # 숫자형 데이터만 선택 & 결측값은 0으로 채움 (입력 17개)
#print(f"X : {X}")
Y = Y_temp.select_dtypes(include=np.number).fillna(0) # 동일하게 결측값은 0으로 채움
#print(f"Y : {Y}")
combined_df = pd.concat([X, Y], axis=1) # 데이터 다시 병합
print(f"combined_df : {combined_df}")

# ==============
# B. 이상치 제거
# ==============

cols_to_clean = ['Hardness', 'Modulus', 'Resistance', 'ravg', 'dHmix'] # 이상치를 제거할 대상인 5개의 목록
removal_ranges = {} # 이상치 제거 기준의 범위 저장하는 딕셔너리

for col in cols_to_clean:
    if col in combined_df.columns:
        #print(f"combined_df.columns : {combined_df.columns}")
        Q1 = combined_df[col].quantile(0.25)
        Q3 = combined_df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR # 이상치 기준의 하한선
        upper_bound = Q3 + 1.5 * IQR # 이상치 기준의 상한선

        # 범위 저장
        removal_ranges[col] = (lower_bound, upper_bound)

        # 이상치 제거
        combined_df = combined_df[
            (combined_df[col] >= lower_bound) & (combined_df[col] <= upper_bound) # 하한선과 상한선 기준 안에 있는 것들만 골라냄
        ]
        #print(f"combined_df : \n{combined_df}")
        #print("\n")
X_clean= combined_df[X.columns]
Y_clean = combined_df[Y_cols]

print(f"총 5개 컬럼 이상치 제거 후 최종 샘플 수 : {len(combined_df)} 행")
print("\n--- 각 컬럼별 1.5 * IQR 이상치 제거 범위 ---")
for col, (lower, upper) in removal_ranges.items():
    print(f"[{col}]: {lower:.4f} ~ {upper:.4f}")

# ==============
# C. 모델 학습
# ==============






X_cols : ['X', 'Y', 'Al', 'Ti', 'Cr', 'Fe', 'Co', 'Ni', 'Cu', 'Zr', 'Mo', 'W', 'Mn', 'Si', 'Mg', 'Modulus', 'ravg', 'delta', 'dHmix', 'ENavg', 'dEN', 'N', 'Compo', 'Unnamed: 27']
combined_df :       X   Y        Al         Ti        Cr   Fe   Co  Ni         Cu  Zr  ...  \
0    -5  15  55.17364   0.000000  21.79127  0.0  0.0   0   0.000000   0  ...   
1    -5  35  59.25293   0.000000  16.76608  0.0  0.0   0   0.000000   0  ...   
2     5 -35  40.23700   0.000000  41.33300  0.0  0.0   0   0.000000   0  ...   
3     5 -15  44.92450   0.000000  33.65950  0.0  0.0   0   0.000000   0  ...   
4     5   5  48.61100   0.000000  26.39050  0.0  0.0   0   0.000000   0  ...   
...  ..  ..       ...        ...       ...  ...  ...  ..        ...  ..  ...   
6644 -5  31   0.00000  15.020688   0.00000  0.0  0.0   0  84.979312   0  ...   
6645 -5  33   0.00000  14.392917   0.00000  0.0  0.0   0  85.607083   0  ...   
6646 -5  35   0.00000  13.765146   0.00000  0.0  0.0   0  86.234854   0  ...   
6647 -5